In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('D:/Downloads/ETA Predict/Participants Data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

D:/Downloads/ETA Predict/Participants Data\Data_Test.xlsx
D:/Downloads/ETA Predict/Participants Data\Data_Train.xlsx
D:/Downloads/ETA Predict/Participants Data\Sample_Submission.xlsx


# Let's predict the Delivery Time with simple ML

1. Prepare the data
2. Select a model and train
3. Solve the problem

In [2]:
# Libraries
!pip install openpyxl

import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [7]:
# FILE PATH
TRAIN_PATH = "D:/Downloads/ETA Predict/Participants Data/Data_Train.xlsx"
TEST_PATH = "D:\Downloads/ETA Predict/Participants Data/Data_Test.xlsx"

## 1. Prepare the data

Read CSV files using pandas and openpyxl. And then, just take a look.

In [8]:
df = pd.read_excel(TRAIN_PATH)
df.head()

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time
0,ID_6321,"FTI College, Law College Road, Pune","Fast Food, Rolls, Burger, Salad, Wraps",₹200,₹50,3.5,12,4,30 minutes
1,ID_2882,"Sector 3, Marathalli","Ice Cream, Desserts",₹100,₹50,3.5,11,4,30 minutes
2,ID_1595,Mumbai Central,"Italian, Street Food, Fast Food",₹150,₹50,3.6,99,30,65 minutes
3,ID_5929,"Sector 1, Noida","Mughlai, North Indian, Chinese",₹250,₹99,3.7,176,95,30 minutes
4,ID_6123,"Rmz Centennial, I Gate, Whitefield","Cafe, Beverages",₹200,₹99,3.2,521,235,65 minutes


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11094 entries, 0 to 11093
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Restaurant     11094 non-null  object
 1   Location       11094 non-null  object
 2   Cuisines       11094 non-null  object
 3   Average_Cost   11094 non-null  object
 4   Minimum_Order  11094 non-null  object
 5   Rating         11094 non-null  object
 6   Votes          11094 non-null  object
 7   Reviews        11094 non-null  object
 8   Delivery_Time  11094 non-null  object
dtypes: object(9)
memory usage: 780.2+ KB


In [10]:
df.describe()

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time
count,11094,11094,11094,11094,11094,11094,11094,11094,11094
unique,7480,35,2179,26,18,33,1103,761,7
top,ID_7184,"Mico Layout, Stage 2, BTM Layout,Bangalore",North Indian,₹200,₹50,-,-,-,30 minutes
freq,22,947,850,3241,10118,1191,2074,2312,7406


Unlike some csv files, this xlsx files contains all values as `object`. To predict with machine-learning, we need to convert this value to numeric values like `int`, or `float`.

In [11]:
df["Restaurant"]

0        ID_6321
1        ID_2882
2        ID_1595
3        ID_5929
4        ID_6123
          ...   
11089    ID_8067
11090    ID_4620
11091    ID_3392
11092    ID_4115
11093    ID_4417
Name: Restaurant, Length: 11094, dtype: object

`Restaurant` column is about the ID of the restaurants. Because it doesn't look useful that much, I will drop this column.

In [12]:
df["Location"]

0        FTI College, Law College Road, Pune
1                       Sector 3, Marathalli
2                             Mumbai Central
3                            Sector 1, Noida
4         Rmz Centennial, I Gate, Whitefield
                        ...                 
11089          BTM Layout 1, Electronic City
11090                       Sector 14, Noida
11091                               Majestic
11092                   Sector 3, Marathalli
11093                     Sector 63A,Gurgaon
Name: Location, Length: 11094, dtype: object

`Location` column is about the location information with words and commas. For more details, Lets see the unique value.

In [13]:
df["Location"].value_counts()

Mico Layout, Stage 2, BTM Layout,Bangalore          947
D-Block, Sector 63, Noida                           806
Sector 1, Noida                                     791
FTI College, Law College Road, Pune                 584
Delhi University-GTB Nagar                          545
Laxman Vihar Industrial Area, Sector 3A, Gurgoan    493
Sector 14, Noida                                    489
Delhi Administration Flats, Timarpur                472
Mumbai Central                                      459
Rmz Centennial, I Gate, Whitefield                  457
Delhi High Court, India Gate                        411
MG Road, Pune                                       402
Nathan Road, Mangaldas Road, Pune                   372
Sandhurst Road, Mumbai CST Area                     370
Sector 3, Marathalli                                366
Majestic                                            334
Delhi Cantt.                                        326
Yerawada, Pune, Maharashtra                     

There are only 35 locations in `Location` column. Some parent address like `Kolkata` or `Pune` appears more that one time. But just let them all as different classes. At here, We can use `One-hot Encoding`.

In [14]:
df_location = pd.get_dummies(df, columns=["Location"])
df_location.head()

,Restaurant,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time,"Location_BTM Layout 1, Electronic City","Location_Babarpur, New Delhi, Delhi",...,"Location_Raja Bazar, Kolkata","Location_Rmz Centennial, I Gate, Whitefield","Location_Sandhurst Road, Mumbai CST Area","Location_Sector 1, Noida","Location_Sector 14, Noida","Location_Sector 3, Marathalli","Location_Sector 63A,Gurgaon","Location_Tejas Nagar Colony, Wadala West, Mumbai","Location_Tiretti, Kolkata","Location_Yerawada, Pune, Maharashtra"
0,ID_6321,"Fast Food, Rolls, Burger, Salad, Wraps",₹200,₹50,3.5,12,4,30 minutes,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ID_2882,"Ice Cream, Desserts",₹100,₹50,3.5,11,4,30 minutes,0,0,...,0,0,0,0,0,1,0,0,0,0
2,ID_1595,"Italian, Street Food, Fast Food",₹150,₹50,3.6,99,30,65 minutes,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ID_5929,"Mughlai, North Indian, Chinese",₹250,₹99,3.7,176,95,30 minutes,0,0,...,0,0,0,1,0,0,0,0,0,0
4,ID_6123,"Cafe, Beverages",₹200,₹99,3.2,521,235,65 minutes,0,0,...,0,1,0,0,0,0,0,0,0,0


In [15]:
df_location.columns

Index(['Restaurant', 'Cuisines', 'Average_Cost', 'Minimum_Order', 'Rating',
       'Votes', 'Reviews', 'Delivery_Time',
       'Location_BTM Layout 1, Electronic City',
       'Location_Babarpur, New Delhi, Delhi',
       'Location_Chandni Chowk, Kolkata',
       'Location_Chatta Bazaar, Malakpet, Hyderabad',
       'Location_D-Block, Sector 63, Noida',
       'Location_Delhi Administration Flats, Timarpur',
       'Location_Delhi Cantt.', 'Location_Delhi High Court, India Gate',
       'Location_Delhi University-GTB Nagar',
       'Location_Dockyard Road, Mumbai CST Area',
       'Location_FTI College, Law College Road, Pune',
       'Location_Gora Bazar, Rajbari, North Dumdum, Kolkata',
       'Location_Hyderabad Public School, Begumpet',
       'Location_Jaya Nagar, Saidabad, Hyderabad',
       'Location_Laxman Vihar Industrial Area, Sector 3A, Gurgoan',
       'Location_MG Road, Pune', 'Location_Majestic',
       'Location_Mico Layout, Stage 2, BTM Layout,Bangalore',
       'Locati

We can see that the `Location` columns seperate into its values and only get 0 or 1 values. 35 columns are newly added because `Location` column has 35 different unique values.

In [16]:
df["Cuisines"]

0        Fast Food, Rolls, Burger, Salad, Wraps
1                           Ice Cream, Desserts
2               Italian, Street Food, Fast Food
3                Mughlai, North Indian, Chinese
4                               Cafe, Beverages
                          ...                  
11089      Tibetan, Chinese, Continental, Momos
11090                                 Fast Food
11091       South Indian, Chinese, North Indian
11092                              North Indian
11093                              North Indian
Name: Cuisines, Length: 11094, dtype: object

Cuisines are the collection of the foods they sell. Some restaurants have many kinds of cuisines, and some are not. Like `Location` Column, we can just simply make `One-hot encoding` from here. But think about:

Ice cream -> [1, 0, 0]

Desserts -> [0, 1, 0]

Ice cream, Desserts -> [0, 0, 1]

It may over-generate the columns with encoder. So, How about this methods?

ce cream -> [1, 0]

Desserts -> [0, 1]

Ice cream, Desserts -> [1, 1]

For this `multi-labeled One-hot Encoding`, we need to process commas at strings while makeing dummies

In [17]:
df_cuisines = df["Cuisines"].str.get_dummies(", ")
df_cuisines

,Afghan,African,American,Andhra,Arabian,Asian,Assamese,Awadhi,BBQ,Bakery,...,Street Food,Sushi,Tamil,Tea,Tex-Mex,Thai,Tibetan,Turkish,Vietnamese,Wraps
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11089,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
11090,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11091,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11092,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
df_cuisines.columns

Index(['Afghan', 'African', 'American', 'Andhra', 'Arabian', 'Asian',
       'Assamese', 'Awadhi', 'BBQ', 'Bakery',
       ...
       'Street Food', 'Sushi', 'Tamil', 'Tea', 'Tex-Mex', 'Thai', 'Tibetan',
       'Turkish', 'Vietnamese', 'Wraps'],
      dtype='object', length=101)

In [19]:
df["Average_Cost"]

0        ₹200
1        ₹100
2        ₹150
3        ₹250
4        ₹200
         ... 
11089    ₹250
11090    ₹100
11091    ₹100
11092    ₹100
11093    ₹100
Name: Average_Cost, Length: 11094, dtype: object

In [20]:
df["Average_Cost"].value_counts()

₹200      3241
₹100      2557
₹150      2462
₹250       881
₹300       537
₹350       283
₹400       282
₹50        265
₹600       154
₹500       101
₹450        63
₹550        60
₹650        55
₹800        44
₹750        38
₹900        15
₹700        15
₹1,000      12
₹850        12
₹1,200       8
₹950         4
₹1,150       1
for          1
₹1,100       1
₹1,400       1
₹2,050       1
Name: Average_Cost, dtype: int64

`Average_Cost` column is about the Average cost of the delivery. It is basically the `int` values with the prefix `₹`. So we need to remove the prefix. And there are data with comma like `1,000`, `1,500`. Also we need to remove the rows that has `for` value. Why `for`? Maybe there's some error. And then, we need to convert to `int`

In [21]:
df_average_cost = df[df["Average_Cost"] != "for"]
df_average_cost = df_average_cost["Average_Cost"].str.replace("₹", "")
df_average_cost = df_average_cost.str.replace(",", "")
df_average_cost = pd.to_numeric(df_average_cost)
df_average_cost

0        200
1        100
2        150
3        250
4        200
        ... 
11089    250
11090    100
11091    100
11092    100
11093    100
Name: Average_Cost, Length: 11093, dtype: int64

In [22]:
df["Minimum_Order"]

0        ₹50
1        ₹50
2        ₹50
3        ₹99
4        ₹99
        ... 
11089    ₹50
11090    ₹50
11091    ₹50
11092    ₹50
11093    ₹50
Name: Minimum_Order, Length: 11094, dtype: object

In [23]:
df["Minimum_Order"].value_counts()

₹50     10118
₹99       779
₹0        158
₹199        8
₹200        8
₹299        3
₹59         3
₹350        3
₹79         2
₹90         2
₹450        2
₹300        2
₹400        1
₹250        1
₹150        1
₹240        1
₹89         1
₹500        1
Name: Minimum_Order, dtype: int64

`Minimum_Order` column is similar with the `Average_Cost` column.

In [24]:
df_minimum_order = df.copy()
minimum_order = df_minimum_order["Minimum_Order"].str.replace("₹", "")
minimum_order = pd.to_numeric(minimum_order)
minimum_order

0        50
1        50
2        50
3        99
4        99
         ..
11089    50
11090    50
11091    50
11092    50
11093    50
Name: Minimum_Order, Length: 11094, dtype: int64

In [25]:
df["Rating"]

0        3.5
1        3.5
2        3.6
3        3.7
4        3.2
        ... 
11089    4.2
11090    3.6
11091    3.5
11092    3.1
11093    NEW
Name: Rating, Length: 11094, dtype: object

In [26]:
df["Rating"].value_counts()

-                     1191
3.7                    869
3.6                    846
3.5                    818
3.8                    800
NEW                    758
3.9                    749
3.4                    718
3.3                    675
4.0                    614
3.2                    511
4.1                    459
3.1                    411
3.0                    302
4.2                    272
4.3                    247
2.9                    199
2.8                    157
4.4                    142
4.5                     78
2.7                     76
2.6                     42
4.6                     41
4.7                     36
2.5                     27
2.4                     13
4.8                     13
Opening Soon            12
4.9                      8
2.3                      6
Temporarily Closed       2
2.2                      1
2.1                      1
Name: Rating, dtype: int64

`Rating` column is basically `float` types but there are some strings, `-`, `NEW`, `Opening Soon` and `Temporarily Closed`. They counts roughly 2k, so rather remove the rows, let's convert the values into `median`

In [27]:
rating_str = ["-", "NEW", "Opening Soon", "Temporarily Closed"]
df_rating_num = df[~df["Rating"].isin(rating_str)]
rating_num = pd.to_numeric(df_rating_num["Rating"])
median = rating_num.median()
median

3.6

In [28]:
df_rating = df.copy()
df_rating.loc[df["Rating"].isin(rating_str), "Rating"] = str(median)
df_rating["Rating"] = pd.to_numeric(df_rating["Rating"])
df_rating["Rating"]

0        3.5
1        3.5
2        3.6
3        3.7
4        3.2
        ... 
11089    4.2
11090    3.6
11091    3.5
11092    3.1
11093    3.6
Name: Rating, Length: 11094, dtype: float64

In [29]:
df["Votes"]

0         12
1         11
2         99
3        176
4        521
        ... 
11089    326
11090     36
11091     45
11092     24
11093      -
Name: Votes, Length: 11094, dtype: object

`Votes` column has only `-`, and the others are all `float` types. for `Votes`, I assume that the default value for votes is 0. No one votes. This is the different.

In [30]:
df_votes = df.copy()
df_votes.loc[df["Votes"] == '-', "Votes"] = str(0)
df_votes["Votes"] = pd.to_numeric(df_votes["Votes"])
df_votes["Votes"]

0         12
1         11
2         99
3        176
4        521
        ... 
11089    326
11090     36
11091     45
11092     24
11093      0
Name: Votes, Length: 11094, dtype: int64

In [31]:
df_votes["Votes"].value_counts()

0       2074
4        248
6        200
7        182
9        181
        ... 
1106       1
935        1
663        1
1116       1
3700       1
Name: Votes, Length: 1103, dtype: int64

In [32]:
df["Reviews"]

0          4
1          4
2         30
3         95
4        235
        ... 
11089    189
11090     16
11091     18
11092      9
11093      -
Name: Reviews, Length: 11094, dtype: object

In [33]:
df["Reviews"].value_counts()

-       2312
2        420
3        387
1        381
4        356
        ... 
1501       1
983        1
343        1
397        1
549        1
Name: Reviews, Length: 761, dtype: int64

`Reviews` column has only `-`, and the others are all `int` types. for `Reviews`, I assume that the default value for reviews is 0. No one reviews.

In [34]:
df_reviews = df.copy()
df_reviews.loc[df["Reviews"] == '-', "Reviews"] = str(0)
df_reviews["Reviews"] = pd.to_numeric(df_reviews["Reviews"])
df_reviews["Reviews"]

0          4
1          4
2         30
3         95
4        235
        ... 
11089    189
11090     16
11091     18
11092      9
11093      0
Name: Reviews, Length: 11094, dtype: int64

In [35]:
df_reviews["Reviews"].value_counts()

0       2312
2        420
3        387
1        381
4        356
        ... 
1501       1
983        1
343        1
397        1
549        1
Name: Reviews, Length: 761, dtype: int64

In [36]:
df["Delivery_Time"]

0        30 minutes
1        30 minutes
2        65 minutes
3        30 minutes
4        65 minutes
            ...    
11089    30 minutes
11090    30 minutes
11091    30 minutes
11092    30 minutes
11093    30 minutes
Name: Delivery_Time, Length: 11094, dtype: object

In [37]:
df["Delivery_Time"].value_counts()

30 minutes     7406
45 minutes     2665
65 minutes      923
120 minutes      62
20 minutes       20
80 minutes       14
10 minutes        4
Name: Delivery_Time, dtype: int64

`Delivery_Time` column is the target value we try to predict. For regression, we should convert the `object` value to `int`.

In [38]:
df_delivery_time = df.copy()
delivery_time = df_delivery_time["Delivery_Time"].str.replace(" minutes", "")
delivery_time = pd.to_numeric(delivery_time)
delivery_time

0        30
1        30
2        65
3        30
4        65
         ..
11089    30
11090    30
11091    30
11092    30
11093    30
Name: Delivery_Time, Length: 11094, dtype: int64

In [39]:
delivery_time.value_counts()

30     7406
45     2665
65      923
120      62
20       20
80       14
10        4
Name: Delivery_Time, dtype: int64

Let's transform all this sequence with `pipeline`.

In [40]:
def copy(df):
    return df.copy()

def drop(df, col):
    new_df = df.drop(col, axis=1)
    return new_df

def drop_row(df, col, find):
    new_df = df[df[col] != find]
    return new_df

def get_dummies(df, col):
    return pd.get_dummies(df, columns=[col])

def get_dummies_comma(df, col):
    new_df = df[col].str.get_dummies(", ")
    new_df = pd.concat([df, new_df], axis = 1)
    new_df.drop("Cuisines", axis=1, inplace=True)
    return new_df

def replace(df, col):
    replace_list = ["₹", ",", " minutes"]
    new_df = df.copy()
    for rep in replace_list:
        new_df[col] = new_df[col].str.replace(rep, "")
    return new_df

def to_numeric(df, col):
    new_df = df.copy()
    new_df[col] = pd.to_numeric(new_df[col])
    return new_df

def fill(df, col, filled):
    find = ["-", "NEW", "Opening Soon", "Temporarily Closed"]
    if filled == 'median':
        num = df[~df[col].isin(find)]
        num = pd.to_numeric(num[col])
        filled = str(num.median())
    new_df = df.copy()
    new_df.loc[df[col].isin(find), col] = filled
    return new_df

def train_pipeline(df):
    return (df.pipe(replace, "Delivery_Time")
              .pipe(to_numeric, "Delivery_Time"))
    
def pipeline(df):
    return (df.pipe(copy)
            .pipe(drop, "Restaurant")
            .pipe(get_dummies, "Location")
            .pipe(get_dummies_comma, "Cuisines")
            .pipe(drop_row, "Average_Cost", "for")
            .pipe(replace, "Average_Cost")
            .pipe(to_numeric, "Average_Cost")
            .pipe(replace, "Minimum_Order")
            .pipe(to_numeric, "Minimum_Order")
            .pipe(fill, "Reviews", "0")
            .pipe(to_numeric, "Reviews")
            .pipe(fill, "Votes", "0")
            .pipe(to_numeric, "Votes")
            .pipe(fill, "Rating", "median")
            .pipe(to_numeric, "Rating"))

train = pipeline(df)
train = train_pipeline(train)
train

,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time,"Location_BTM Layout 1, Electronic City","Location_Babarpur, New Delhi, Delhi","Location_Chandni Chowk, Kolkata","Location_Chatta Bazaar, Malakpet, Hyderabad",...,Street Food,Sushi,Tamil,Tea,Tex-Mex,Thai,Tibetan,Turkish,Vietnamese,Wraps
0,200,50,3.5,12,4,30,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,100,50,3.5,11,4,30,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,150,50,3.6,99,30,65,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,250,99,3.7,176,95,30,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,200,99,3.2,521,235,65,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11089,250,50,4.2,326,189,30,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
11090,100,50,3.6,36,16,30,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11091,100,50,3.5,45,18,30,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11092,100,50,3.1,24,9,30,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Train data

In [41]:
train_data = drop(train, "Delivery_Time")
train_data

,Average_Cost,Minimum_Order,Rating,Votes,Reviews,"Location_BTM Layout 1, Electronic City","Location_Babarpur, New Delhi, Delhi","Location_Chandni Chowk, Kolkata","Location_Chatta Bazaar, Malakpet, Hyderabad","Location_D-Block, Sector 63, Noida",...,Street Food,Sushi,Tamil,Tea,Tex-Mex,Thai,Tibetan,Turkish,Vietnamese,Wraps
0,200,50,3.5,12,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,100,50,3.5,11,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,150,50,3.6,99,30,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,250,99,3.7,176,95,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,200,99,3.2,521,235,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11089,250,50,4.2,326,189,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
11090,100,50,3.6,36,16,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11091,100,50,3.5,45,18,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11092,100,50,3.1,24,9,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


`Pandas Dataframe` should be converted into `numpy array` to make a model

In [42]:
train_data_numpy = train_data.to_numpy()
train_data_numpy

array([[200. ,  50. ,   3.5, ...,   0. ,   0. ,   1. ],
       [100. ,  50. ,   3.5, ...,   0. ,   0. ,   0. ],
       [150. ,  50. ,   3.6, ...,   0. ,   0. ,   0. ],
       ...,
       [100. ,  50. ,   3.5, ...,   0. ,   0. ,   0. ],
       [100. ,  50. ,   3.1, ...,   0. ,   0. ,   0. ],
       [100. ,  50. ,   3.6, ...,   0. ,   0. ,   0. ]])

In [43]:
train_label = train["Delivery_Time"]
train_label

0        30
1        30
2        65
3        30
4        65
         ..
11089    30
11090    30
11091    30
11092    30
11093    30
Name: Delivery_Time, Length: 11093, dtype: int64

## 2. Select a model and train

Now with prepared data, I will train the model with simple `Scikit-Learn` based Machine-Learning. I'll use `RandomForestRegressor` which is very powerful for regression.

In [44]:
forest = RandomForestRegressor(n_estimators=10, random_state=42)
forest.fit(train_data_numpy, train_label)

RandomForestRegressor(n_estimators=10, random_state=42)

Let's check the error using cross validation especially the `K-fold`. The `cross_val_score` split the data into `train data` and `validation data`. `K` means we loop this things for `k` times. Set `k = 10`.

In [45]:
forest_scores = cross_val_score(forest, train_data_numpy, train_label, scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())
    
display_scores(forest_rmse_scores)

Scores: [10.002267    9.54095354  9.46107189 10.36611001 10.04652359 10.2903205
 10.06636721 10.15290653 11.01493319 10.78654626]
Mean: 10.172799971500904
Standard deviation: 0.4582128759317515


RMSE mean for `10-fold cross validation`: 10.172799971500904

## 3. Solve the problem

Now get the test data and check the model precision.

In [46]:
pd.read_excel(TEST_PATH).head()

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews
0,ID_2842,"Mico Layout, Stage 2, BTM Layout,Bangalore","North Indian, Chinese, Assamese",₹350,₹50,4.2,361,225
1,ID_730,"Mico Layout, Stage 2, BTM Layout,Bangalore","Biryani, Kebab",₹100,₹50,NEW,-,-
2,ID_4620,"Sector 1, Noida",Fast Food,₹100,₹50,3.6,36,16
3,ID_5470,"Babarpur, New Delhi, Delhi","Mithai, North Indian, Chinese, Fast Food, Sout...",₹200,₹50,3.6,66,33
4,ID_3249,"Sector 1, Noida","Chinese, Fast Food",₹150,₹50,2.9,38,14


In [43]:
test_df = pd.read_excel(TEST_PATH)

dataset = pd.concat([df, test_df], axis=0)
dataset = pipeline(dataset)

test = dataset[len(train):] # for 1-removed row
test = test.drop("Delivery_Time", axis=1)

prediction = forest.predict(test)
                            
submission = pd.DataFrame({"Delivery_Time": prediction})
submission.to_csv('submission.csv', index=False)